In [153]:
import requests
import pandas as pd
from datetime import datetime

Totaal: 816232+1 moties

Select start and end years:

In [166]:
start_year = '2010'
end_year = '2011'

In [167]:
url = f"https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Stemming?$filter= year(GewijzigdOp) ge {start_year} and year(GewijzigdOp) le {end_year} & orderby=GewijzigdOp "
response = requests.get(url)
data = response.json()
data

{'@odata.context': 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/$metadata#Stemming',
 'value': [{'Id': '2dcf05a9-6ec8-4f70-b3b0-0006f1f648b9',
   'Besluit_Id': '2c68bf5d-d41b-4333-bcef-762570f14b34',
   'Soort': 'Voor',
   'FractieGrootte': 30,
   'ActorNaam': 'PvdA',
   'ActorFractie': 'PvdA',
   'Vergissing': False,
   'SidActorLid': None,
   'SidActorFractie': 'S-1-365867521-2120874753-2900001417-1207812138-1581088948-1481602963',
   'Persoon_Id': None,
   'Fractie_Id': '8d46d23c-4f20-49be-b279-5439a2ef8d17',
   'GewijzigdOp': '2010-09-29T11:21:36.007+02:00',
   'ApiGewijzigdOp': '2019-07-26T13:03:47.4099109Z',
   'Verwijderd': False},
  {'Id': '3cece3b0-d9bc-4280-bfe3-000da364a688',
   'Besluit_Id': '5dafd6e1-07e3-4990-9245-40108d8e704e',
   'Soort': 'Voor',
   'FractieGrootte': 31,
   'ActorNaam': 'VVD',
   'ActorFractie': 'VVD',
   'Vergissing': False,
   'SidActorLid': None,
   'SidActorFractie': 'S-1-365867521-2120874753-2712795160-1237583058-2671294620-1237654743',
  

In [169]:
start_year = '2010'
end_year = '2011'

# Define the initial URL
url = url = f"https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Stemming?$filter= year(GewijzigdOp) ge {start_year} and year(GewijzigdOp) le {end_year} & orderby=GewijzigdOp "

# Initialize an empty list to store all the data points
all_data = []

# Loop to fetch all pages of data
while url:
    # Make an HTTP GET request to the current URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the JSON response to a Python dictionary
        data = response.json()
        
        # Extract the 'value' part of the JSON response and append it to the list
        all_data.extend(data['value'])
        
        # Check if there's a next page link
        if '@odata.nextLink' in data:
            # Update the URL to fetch the next page of data
            url = data['@odata.nextLink']
        else:
            # If there's no next page link, exit the loop
            url = None
    else:
        # If the request was not successful, print an error message and exit the loop
        print("Error:", response.status_code)
        url = None

# Convert the list of data points to a Pandas DataFrame
df = pd.DataFrame(all_data)


In [170]:
df.sort_values(by='GewijzigdOp')

,Id,Besluit_Id,Soort,FractieGrootte,ActorNaam,ActorFractie,Vergissing,SidActorLid,SidActorFractie,Persoon_Id,Fractie_Id,GewijzigdOp,ApiGewijzigdOp,Verwijderd
4959,1a805087-1b8f-400d-a83d-f2a002704f98,8890847e-f3d6-4131-a09c-8bc989fa580c,Voor,41,CDA,CDA,False,None,S-1-365867521-2120874753-3726368973-1295802169...,None,62c1a13c-85ff-40ed-90f7-a9546d61f869,2010-01-20T10:30:34.273+01:00,2019-07-26T12:58:44.7374442Z,False
2229,65b900b6-ecd5-4a74-9895-6baadbf515cb,8890847e-f3d6-4131-a09c-8bc989fa580c,Voor,33,PvdA,PvdA,False,None,S-1-365867521-2120874753-2900001417-1207812138...,None,8d46d23c-4f20-49be-b279-5439a2ef8d17,2010-01-20T10:30:34.29+01:00,2019-07-26T13:03:44.4724072Z,False
627,bc98b615-7630-4571-bef2-1ef260534d40,8890847e-f3d6-4131-a09c-8bc989fa580c,Voor,25,SP,SP,False,None,S-1-365867521-2120874753-3918075670-1133036099...,None,a3689bb6-3914-4d5c-a6a8-42e24582e299,2010-01-20T10:30:34.307+01:00,2019-07-26T13:02:52.3787956Z,False
2178,6e5d14fb-a67a-4f5c-8c5d-6950542d34d6,8890847e-f3d6-4131-a09c-8bc989fa580c,Voor,7,GroenLinks,GroenLinks,False,None,S-1-365867521-2120874753-2386850707-1282307975...,None,8fd1a907-0355-4d27-8dc1-fd5a531b471e,2010-01-20T10:30:34.32+01:00,2019-07-26T13:04:01.5505616Z,False
3138,00964b2e-8f80-467f-8a78-9903b64bbca0,8890847e-f3d6-4131-a09c-8bc989fa580c,Voor,9,PVV,PVV,False,None,S-1-365867521-2120874753-203711267-1259409971-...,None,65129918-f256-4975-9da4-488da34d6695,2010-01-20T10:30:34.32+01:00,2019-07-19T11:51:15.137479Z,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,c662e244-553b-4dc0-867b-376c269ed807,760bd3a7-c76d-4f27-a5d1-08e03d43c79c,Voor,30,PvdA,PvdA,False,None,S-1-365867521-2120874753-2900001417-1207812138...,None,8d46d23c-4f20-49be-b279-5439a2ef8d17,2011-12-27T12:00:37.81+01:00,2019-07-26T13:03:45.769285Z,False
3056,b7725ba1-1a7b-4f08-96e0-9492f428fed6,760bd3a7-c76d-4f27-a5d1-08e03d43c79c,Voor,24,PVV,PVV,False,None,S-1-365867521-2120874753-203711267-1259409971-...,None,65129918-f256-4975-9da4-488da34d6695,2011-12-27T12:00:37.827+01:00,2019-07-19T11:51:15.1062344Z,False
1540,6bba8581-24fd-4062-a55e-4c339a1ead1d,760bd3a7-c76d-4f27-a5d1-08e03d43c79c,Voor,2,SGP,SGP,False,None,S-1-365867521-2120874753-1520352372-1191064225...,None,77f9b6f1-b1a9-4d1b-a05e-9936e79d8fa5,2011-12-27T12:00:37.827+01:00,2019-07-26T13:03:37.706763Z,False
2868,ec5b4848-af99-43f7-bd02-8b3d525d848a,760bd3a7-c76d-4f27-a5d1-08e03d43c79c,Voor,15,SP,SP,False,None,S-1-365867521-2120874753-3918075670-1133036099...,None,a3689bb6-3914-4d5c-a6a8-42e24582e299,2011-12-27T12:00:37.857+01:00,2019-07-26T13:02:51.9098065Z,False


In [171]:
df.shape

(5246, 14)